# PAMTRA simulations radiosondes
Simulate radiosonde profiles of 2019 at four locations with PAMTRA for MWI bandpass effect estimation. Originally, an emissivity of 0.6 was assumed, because the default emissivity in PAMTRA was not changed. That value is much lower than the 0.85-0.9 in TESSEM2 for ocean surfaces at 183 GHz for conical scanners under vertical polarization like MWI. Changing the surface emissivity will change the spectral TB variation. The very low TB observed at the wings of the absorption line are about 50 K warmer compared to the simulations with an emissivity of 0.6. The previous script was completely integrated in here, and deleted to avoid confusion.

- Input: Radiosonde data (txt files), frequency list (txt file)
- Output: TB for radiosondes (one file with all angles and both polarization for completeness, one file with only V and upward radiance - V and H are the same anyway!)

The smaller output file is copied to the home directory to be compatible with the down-stream tasks.

Further Idea: run this for different emissivities and see how the surface emissivity influences the bandpass effect. It might be similar to what clouds do, except only for low IWV.

Latest change: 2022-08-02

In [1]:
from __future__ import division
import pyPamtra

import sys
import os

import numpy as np
import pandas as pd
import xarray as xr

import datetime
from glob import glob

sys.path.append(os.environ['PATH_PHD']+'/projects/pamtra_model/tools')
from pamtra_tools import pam_to_xarray

In [2]:
# find all radiosonde files
files = glob(os.environ['PATH_PHD']+'/projects/mwi_bandpass_effects/data/atmosphere/2019/*/*/*.txt')

In [3]:
# ID for each radiosonde
rsd_profiles = np.array([file[-25:-4] for file in files])
lst_ds_pam = []  # store PAMTRA simulation corresponding the rsd_profile

for i, file in enumerate(files):
    
    print(f'{i+1}/{len(files)} - {file}')
    
    # read radiosonde profile
    df_rsd = pd.read_csv(file, comment='#')
    df_rsd.drop(df_rsd[df_rsd['z [m]'] < 0].index, inplace=True)  # remove entries with negative height
    df_rsd.dropna(axis='index', subset=['p [hPa]', 'z [m]', 'T [C]', 'RH [%]'], inplace=True)
    
    pam = pyPamtra.pyPamtra()

    pam.nmlSet['creator'] = 'Nils Risse'

    pam.nmlSet['active'] = False
    pam.nmlSet['passive'] = True
    pam.nmlSet['outpol'] = 'VH'
    pam.set['freqs'] = np.loadtxt(os.environ['PATH_PHD']+'/projects/mwi_bandpass_effects/data/frequencies.txt')

    pam.nmlSet['add_obs_height_to_layer'] = False  # not for satellite view

    pam.nmlSet['gas_mod'] = 'R98'
    pam.nmlSet['emissivity'] = 0.9
    
    # create pamtra profile based on radiosonde data
    pam_profile = dict()
    
    pam_profile['ngridx'] = 1
    pam_profile['ngridy'] = 1
    
    pam_profile['obs_height'] = np.array([[[833000]]])

    # time (not important)
    time = np.datetime64('2022-01-01').astype('datetime64[s]').astype('int')
    pam_profile['timestamp'] = np.array([[time]])

    # location (not important)
    pam_profile['lat'] = np.array([[79]])
    pam_profile['lon'] = np.array([[0]])

    # physical properties (only groundtemp important for fixed emissivity)
    pam_profile['groundtemp'] = np.array([[np.nan]])  # use lowest atmospheric temperature
    pam_profile['sfc_salinity'] = np.array([[33]])
    pam_profile['wind10u'] = np.array([[0]])
    pam_profile['wind10v'] = np.array([[0]])
    pam_profile['sfc_slf'] = np.array([[1]])
    pam_profile['sfc_sif'] = np.array([[0]])
    
    # atmosphere
    pam_profile["press"] = df_rsd['p [hPa]'] * 100
    pam_profile["hgt"] = df_rsd['z [m]']
    pam_profile["temp"] = df_rsd['T [C]'] + 273.15
    pam_profile["relhum"] = df_rsd['RH [%]']
    
    # radiative transfer
    pam_profile['sfc_type'] = np.array([[-1]])  # fixed emissivity
    pam_profile['sfc_model'] = np.array([[-1]])  # not used, because emissivity fixed
    pam_profile['sfc_refl'] = np.array([['S']])  # specular

    pam.createProfile(**pam_profile)

    pam.df.addHydrometeor(('ice', -99., -1, 917., 130., 3.0, 0.684, 2., 3, 1,
                           'mono_cosmo_ice', -99., -99., -99., -99., -99., -99.,
                           'mie-sphere', 'heymsfield10_particles', 0.0))
    
    pam.runParallelPamtra(pam.set['freqs'],
                      pp_deltaX=1, 
                      pp_deltaY=1, 
                      pp_deltaF=10, 
                      pp_local_workers="auto")
    pam.addIntegratedValues()
    
    lst_ds_pam.append(pam_to_xarray(pam))

ds_pam = xr.concat(
    lst_ds_pam,
    pd.Index(rsd_profiles, name='profile'))

1/1341 - /home/nrisse/phd/projects/mwi_bandpass_effects/data/atmosphere/2019/01/01/ID_01004_201901011200.txt


/home/nrisse/lib/python/pyPamtra/core.py:716: UserWarning: No descriptor file defined. Assuming dry profile without hydrometeors
  warnings.warn("No descriptor file defined. Assuming dry profile without hydrometeors")
/home/nrisse/lib/python/pyPamtra/core.py:881: Warning: hydro_q set to 0
  warnings.warn(qValue + " set to 0", Warning)
/home/nrisse/lib/python/pyPamtra/core.py:881: Warning: hydro_reff set to 0
  warnings.warn(qValue + " set to 0", Warning)
/home/nrisse/lib/python/pyPamtra/core.py:881: Warning: hydro_n set to 0
  warnings.warn(qValue + " set to 0", Warning)
/home/nrisse/lib/python/pyPamtra/core.py:892: Warning: airturb set to nan
  warnings.warn(qValue + " set to nan", Warning)
/home/nrisse/lib/python/pyPamtra/core.py:892: Warning: wind_w set to nan
  warnings.warn(qValue + " set to nan", Warning)
/home/nrisse/lib/python/pyPamtra/core.py:892: Warning: wind_uv set to nan
  warnings.warn(qValue + " set to nan", Warning)
/home/nrisse/lib/python/pyPamtra/core.py:892: Warning:

2/1341 - /home/nrisse/phd/projects/mwi_bandpass_effects/data/atmosphere/2019/01/01/ID_48698_201901011200.txt
3/1341 - /home/nrisse/phd/projects/mwi_bandpass_effects/data/atmosphere/2019/01/01/ID_10410_201901011200.txt
4/1341 - /home/nrisse/phd/projects/mwi_bandpass_effects/data/atmosphere/2019/01/01/ID_78954_201901011200.txt
5/1341 - /home/nrisse/phd/projects/mwi_bandpass_effects/data/atmosphere/2019/01/02/ID_01004_201901021200.txt
6/1341 - /home/nrisse/phd/projects/mwi_bandpass_effects/data/atmosphere/2019/01/02/ID_48698_201901021200.txt
7/1341 - /home/nrisse/phd/projects/mwi_bandpass_effects/data/atmosphere/2019/01/02/ID_10410_201901021200.txt
8/1341 - /home/nrisse/phd/projects/mwi_bandpass_effects/data/atmosphere/2019/01/02/ID_78954_201901021200.txt
9/1341 - /home/nrisse/phd/projects/mwi_bandpass_effects/data/atmosphere/2019/01/03/ID_01004_201901031200.txt
10/1341 - /home/nrisse/phd/projects/mwi_bandpass_effects/data/atmosphere/2019/01/03/ID_48698_201901031200.txt
11/1341 - /home/nr

77/1341 - /home/nrisse/phd/projects/mwi_bandpass_effects/data/atmosphere/2019/01/21/ID_78954_201901211200.txt
78/1341 - /home/nrisse/phd/projects/mwi_bandpass_effects/data/atmosphere/2019/01/22/ID_01004_201901221200.txt
79/1341 - /home/nrisse/phd/projects/mwi_bandpass_effects/data/atmosphere/2019/01/22/ID_48698_201901221200.txt
80/1341 - /home/nrisse/phd/projects/mwi_bandpass_effects/data/atmosphere/2019/01/22/ID_10410_201901221200.txt
81/1341 - /home/nrisse/phd/projects/mwi_bandpass_effects/data/atmosphere/2019/01/22/ID_78954_201901221200.txt
82/1341 - /home/nrisse/phd/projects/mwi_bandpass_effects/data/atmosphere/2019/01/23/ID_01004_201901231200.txt
83/1341 - /home/nrisse/phd/projects/mwi_bandpass_effects/data/atmosphere/2019/01/23/ID_48698_201901231200.txt
84/1341 - /home/nrisse/phd/projects/mwi_bandpass_effects/data/atmosphere/2019/01/23/ID_10410_201901231200.txt
85/1341 - /home/nrisse/phd/projects/mwi_bandpass_effects/data/atmosphere/2019/01/23/ID_78954_201901231200.txt
86/1341 - 

152/1341 - /home/nrisse/phd/projects/mwi_bandpass_effects/data/atmosphere/2019/02/09/ID_01004_201902091200.txt
153/1341 - /home/nrisse/phd/projects/mwi_bandpass_effects/data/atmosphere/2019/02/09/ID_48698_201902091200.txt
154/1341 - /home/nrisse/phd/projects/mwi_bandpass_effects/data/atmosphere/2019/02/09/ID_10410_201902091200.txt
155/1341 - /home/nrisse/phd/projects/mwi_bandpass_effects/data/atmosphere/2019/02/09/ID_78954_201902091200.txt
156/1341 - /home/nrisse/phd/projects/mwi_bandpass_effects/data/atmosphere/2019/02/10/ID_01004_201902101200.txt
157/1341 - /home/nrisse/phd/projects/mwi_bandpass_effects/data/atmosphere/2019/02/10/ID_48698_201902101200.txt
158/1341 - /home/nrisse/phd/projects/mwi_bandpass_effects/data/atmosphere/2019/02/10/ID_10410_201902101200.txt
159/1341 - /home/nrisse/phd/projects/mwi_bandpass_effects/data/atmosphere/2019/02/10/ID_78954_201902101200.txt
160/1341 - /home/nrisse/phd/projects/mwi_bandpass_effects/data/atmosphere/2019/02/11/ID_01004_201902111200.txt
1

...

1040/1341 - /home/nrisse/phd/projects/mwi_bandpass_effects/data/atmosphere/2019/10/10/ID_10410_201910101200.txt
1041/1341 - /home/nrisse/phd/projects/mwi_bandpass_effects/data/atmosphere/2019/10/11/ID_01004_201910111200.txt
1042/1341 - /home/nrisse/phd/projects/mwi_bandpass_effects/data/atmosphere/2019/10/11/ID_48698_201910111200.txt
1043/1341 - /home/nrisse/phd/projects/mwi_bandpass_effects/data/atmosphere/2019/10/11/ID_10410_201910111200.txt
1044/1341 - /home/nrisse/phd/projects/mwi_bandpass_effects/data/atmosphere/2019/10/12/ID_01004_201910121200.txt
1045/1341 - /home/nrisse/phd/projects/mwi_bandpass_effects/data/atmosphere/2019/10/12/ID_48698_201910121200.txt
1046/1341 - /home/nrisse/phd/projects/mwi_bandpass_effects/data/atmosphere/2019/10/12/ID_10410_201910121200.txt
1047/1341 - /home/nrisse/phd/projects/mwi_bandpass_effects/data/atmosphere/2019/10/13/ID_01004_201910131200.txt
1048/1341 - /home/nrisse/phd/projects/mwi_bandpass_effects/data/atmosphere/2019/10/13/ID_48698_201910131

1114/1341 - /home/nrisse/phd/projects/mwi_bandpass_effects/data/atmosphere/2019/11/03/ID_78954_201911031200.txt
1115/1341 - /home/nrisse/phd/projects/mwi_bandpass_effects/data/atmosphere/2019/11/04/ID_01004_201911041200.txt
1116/1341 - /home/nrisse/phd/projects/mwi_bandpass_effects/data/atmosphere/2019/11/04/ID_48698_201911041200.txt
1117/1341 - /home/nrisse/phd/projects/mwi_bandpass_effects/data/atmosphere/2019/11/04/ID_10410_201911041200.txt
1118/1341 - /home/nrisse/phd/projects/mwi_bandpass_effects/data/atmosphere/2019/11/04/ID_78954_201911041200.txt
1119/1341 - /home/nrisse/phd/projects/mwi_bandpass_effects/data/atmosphere/2019/11/05/ID_01004_201911051200.txt
1120/1341 - /home/nrisse/phd/projects/mwi_bandpass_effects/data/atmosphere/2019/11/05/ID_48698_201911051200.txt
1121/1341 - /home/nrisse/phd/projects/mwi_bandpass_effects/data/atmosphere/2019/11/05/ID_10410_201911051200.txt
1122/1341 - /home/nrisse/phd/projects/mwi_bandpass_effects/data/atmosphere/2019/11/05/ID_78954_201911051

1188/1341 - /home/nrisse/phd/projects/mwi_bandpass_effects/data/atmosphere/2019/11/23/ID_10410_201911231200.txt
1189/1341 - /home/nrisse/phd/projects/mwi_bandpass_effects/data/atmosphere/2019/11/24/ID_01004_201911241200.txt
1190/1341 - /home/nrisse/phd/projects/mwi_bandpass_effects/data/atmosphere/2019/11/24/ID_48698_201911241200.txt
1191/1341 - /home/nrisse/phd/projects/mwi_bandpass_effects/data/atmosphere/2019/11/24/ID_10410_201911241200.txt
1192/1341 - /home/nrisse/phd/projects/mwi_bandpass_effects/data/atmosphere/2019/11/24/ID_78954_201911241200.txt
1193/1341 - /home/nrisse/phd/projects/mwi_bandpass_effects/data/atmosphere/2019/11/25/ID_01004_201911251200.txt
1194/1341 - /home/nrisse/phd/projects/mwi_bandpass_effects/data/atmosphere/2019/11/25/ID_48698_201911251200.txt
1195/1341 - /home/nrisse/phd/projects/mwi_bandpass_effects/data/atmosphere/2019/11/25/ID_10410_201911251200.txt
1196/1341 - /home/nrisse/phd/projects/mwi_bandpass_effects/data/atmosphere/2019/11/25/ID_78954_201911251

1262/1341 - /home/nrisse/phd/projects/mwi_bandpass_effects/data/atmosphere/2019/12/12/ID_78954_201912121200.txt
1263/1341 - /home/nrisse/phd/projects/mwi_bandpass_effects/data/atmosphere/2019/12/13/ID_01004_201912131200.txt
1264/1341 - /home/nrisse/phd/projects/mwi_bandpass_effects/data/atmosphere/2019/12/13/ID_48698_201912131200.txt
1265/1341 - /home/nrisse/phd/projects/mwi_bandpass_effects/data/atmosphere/2019/12/13/ID_10410_201912131200.txt
1266/1341 - /home/nrisse/phd/projects/mwi_bandpass_effects/data/atmosphere/2019/12/13/ID_78954_201912131200.txt
1267/1341 - /home/nrisse/phd/projects/mwi_bandpass_effects/data/atmosphere/2019/12/14/ID_01004_201912141200.txt
1268/1341 - /home/nrisse/phd/projects/mwi_bandpass_effects/data/atmosphere/2019/12/14/ID_48698_201912141200.txt
1269/1341 - /home/nrisse/phd/projects/mwi_bandpass_effects/data/atmosphere/2019/12/14/ID_10410_201912141200.txt
1270/1341 - /home/nrisse/phd/projects/mwi_bandpass_effects/data/atmosphere/2019/12/14/ID_78954_201912141

1336/1341 - /home/nrisse/phd/projects/mwi_bandpass_effects/data/atmosphere/2019/12/31/ID_48698_201912311200.txt
1337/1341 - /home/nrisse/phd/projects/mwi_bandpass_effects/data/atmosphere/2019/12/31/ID_10410_201912311200.txt
1338/1341 - /home/nrisse/phd/projects/mwi_bandpass_effects/data/atmosphere/2019/12/31/ID_78954_201912311200.txt
1339/1341 - /home/nrisse/phd/projects/mwi_bandpass_effects/data/atmosphere/2019/12/05/ID_48698_201912051200.txt
1340/1341 - /home/nrisse/phd/projects/mwi_bandpass_effects/data/atmosphere/2019/12/05/ID_10410_201912051200.txt
1341/1341 - /home/nrisse/phd/projects/mwi_bandpass_effects/data/atmosphere/2019/12/05/ID_78954_201912051200.txt


In [4]:
# write result to file
file_out = os.environ['PATH_SEC']+'/data/mwi_bandpass_effects/TB_radiosondes_2019_complete.nc'
ds_pam.to_netcdf(file_out)

In [5]:
# write result with reduced variables to file
ds_pam_red = ds_pam.sel(grid_x=0, grid_y=0, direction='up', outlevel=0, polarization='V', hydro_class=0)
file_out = os.environ['PATH_SEC']+'/data/mwi_bandpass_effects/TB_radiosondes_2019.nc'
ds_pam_red.to_netcdf(file_out)